In [229]:
import openrouteservice
import folium
import pandas as pd
import geopandas as gpd
import pyspark
from pyspark.sql import SparkSession, functions as F
import json
import requests
import regex

In [231]:
# rent by suburb
rent_by_suburb = pd.read_excel("../data/raw/rent_by_suburb.xlsx", 'All properties')

In [232]:
rent_by_suburb.head()

,Suburbs - All properties,Unnamed: 1,Lease commenced in year ending,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175
0,NaN,NaN,Mar 2000,NaN,Jun 2000,NaN,Sep 2000,NaN,Dec 2000,NaN,...,Sep 2020,NaN,Dec 2020,NaN,Mar 2021,NaN,Jun 2021,NaN,Sep 2021,NaN
1,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
2,Inner Melbourne,Albert Park-Middle Park-West St Kilda,1146,260,1136,260,1177,270,1176,275,...,802,560,848,550,899,500,942,500,940,500
3,NaN,Armadale,729,200,731,200,732,205,732,210,...,709,500,729,495,792,450,820,440,855,425
4,NaN,Carlton North,864,260,814,260,800,265,735,270,...,520,590,547,590,582,580,613,580,612,578


In [233]:
median_cols = []
for i in range(176):
    if i % 2 == 1:
        median_cols.append(i)

In [234]:
rent_by_suburb = rent_by_suburb.iloc[:,median_cols]

In [235]:
# now we want to remove first two rows
rent_by_suburb = rent_by_suburb.drop([rent_by_suburb.index[0]])
rent_by_suburb = rent_by_suburb.drop([rent_by_suburb.index[0]])

In [236]:
#note that every 4 columns is 1 year so need to predict 12 columns, every column hereis the median 
rent_by_suburb.head(10)

,Unnamed: 1,Unnamed: 3,Unnamed: 5,Unnamed: 7,Unnamed: 9,Unnamed: 11,Unnamed: 13,Unnamed: 15,Unnamed: 17,Unnamed: 19,...,Unnamed: 157,Unnamed: 159,Unnamed: 161,Unnamed: 163,Unnamed: 165,Unnamed: 167,Unnamed: 169,Unnamed: 171,Unnamed: 173,Unnamed: 175
2,Albert Park-Middle Park-West St Kilda,260,260,270,275,275,280,280,290,300,...,560,565,595,600,585,560,550,500,500,500
3,Armadale,200,200,205,210,215,220,225,230,235,...,480,480,485,500,500,500,495,450,440,425
4,Carlton North,260,260,265,270,270,275,280,280,290,...,577,580,580,585,590,590,590,580,580,578
5,Carlton-Parkville,260,260,260,260,260,260,265,270,270,...,440,430,420,404,400,400,395,350,340,330
6,CBD-St Kilda Rd,320,320,320,320,320,320,320,320,320,...,500,510,510,510,495,460,420,380,370,350
7,Collingwood-Abbotsford,230,230,240,240,245,250,250,250,250,...,495,495,500,500,490,480,460,450,430,425
8,Docklands,-,340,-,-,-,-,-,-,320,...,580,575,580,570,550,500,460,420,400,400
9,East Melbourne,265,268,280,273,290,295,300,320,320,...,500,520,550,550,550,515,495,450,450,450
10,East St Kilda,180,180,180,185,185,190,195,195,200,...,420,425,425,425,425,420,410,395,390,380
11,Elwood,200,200,200,210,220,220,220,230,230,...,450,450,450,450,450,450,440,425,422,420


In [237]:
rent_by_suburb = rent_by_suburb[rent_by_suburb['Unnamed: 3'] != '-']

In [238]:
rent_by_suburb.rename(columns={'Unnamed: 1':'suburb'}, inplace=True)

In [239]:

rent_by_suburb.drop(rent_by_suburb.index[rent_by_suburb['suburb'] == 'Group Total'], inplace=True)

In [240]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

In [241]:
start_date = date(2000,3,1)

In [242]:
header_list = ['suburb']

In [243]:
for i in range(0, len(median_cols)-1):
    #print(start_date)
    header_list.append(start_date)
    start_date = start_date + relativedelta(months=+3)


In [244]:
header_dict = {}

In [245]:
header_list[0]

'suburb'

In [246]:
header_list[86]

datetime.date(2021, 6, 1)

In [247]:

for i in range(0,len(rent_by_suburb.columns)):
    #print(header_list[i])
    header_dict[rent_by_suburb.columns[i]] = header_list[i]

In [248]:
rent_by_suburb.rename(columns=header_dict,inplace=True)

In [249]:
rent_by_suburb

,suburb,2000-03-01,2000-06-01,2000-09-01,2000-12-01,2001-03-01,2001-06-01,2001-09-01,2001-12-01,2002-03-01,...,2019-06-01,2019-09-01,2019-12-01,2020-03-01,2020-06-01,2020-09-01,2020-12-01,2021-03-01,2021-06-01,2021-09-01
2,Albert Park-Middle Park-West St Kilda,260,260,270,275,275,280,280,290,300,...,560,565,595,600,585,560,550,500,500,500
3,Armadale,200,200,205,210,215,220,225,230,235,...,480,480,485,500,500,500,495,450,440,425
4,Carlton North,260,260,265,270,270,275,280,280,290,...,577,580,580,585,590,590,590,580,580,578
5,Carlton-Parkville,260,260,260,260,260,260,265,270,270,...,440,430,420,404,400,400,395,350,340,330
6,CBD-St Kilda Rd,320,320,320,320,320,320,320,320,320,...,500,510,510,510,495,460,420,380,370,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Traralgon,125,120,125,125,125,125,130,125,130,...,285,290,295,300,300,310,320,330,340,350
156,Wanagaratta,125,125,130,130,130,130,130,135,135,...,290,289,290,290,300,300,310,320,330,350
157,Warragul,130,135,135,135,135,135,140,145,150,...,350,350,348,350,350,360,380,375,380,390
158,Warrnambool,130,135,135,135,140,140,140,145,145,...,320,330,330,340,340,340,350,350,360,370


In [250]:
rent_time_ser = pd.melt(rent_by_suburb, id_vars=['suburb'], value_vars=header_list[1:],var_name='Month', value_name='Median Rent')

In [251]:
rent_time_ser.set_index('Month', inplace = True)


In [252]:
rent_time_ser.astype({'Median Rent':'float'})

,suburb,Median Rent
Month,,
2000-03-01,Albert Park-Middle Park-West St Kilda,260.0
2000-03-01,Armadale,200.0
2000-03-01,Carlton North,260.0
2000-03-01,Carlton-Parkville,260.0
2000-03-01,CBD-St Kilda Rd,320.0
...,...,...
2021-09-01,Traralgon,350.0
2021-09-01,Wanagaratta,350.0
2021-09-01,Warragul,390.0


In [263]:
rent_time_ser['no_id'] = 0

In [264]:
for i in range(0,len(rent_time_ser)):
    rent_time_ser['no_id'][i] = i 

/tmp/ipykernel_2643/2598705627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_time_ser['no_id'][i] = i


In [269]:
type(rent_time_ser['no_id'][0])

numpy.int64